In [48]:
from bs4 import BeautifulSoup
import urllib
import urllib.request
import io


In [50]:
url = "https://arxiv.org/find/stat/1/au:+Young_S/0/1/0/all/0/1"

u = urllib.request.urlopen(url, data = None)
f = io.TextIOWrapper(u,encoding='utf-8')
text = f.read()

soup = BeautifulSoup(text, 'html.parser')

In [51]:
soup.find_all(attrs={'class': "primary-subject"})

[<span class="primary-subject">Applications (stat.AP)</span>,
 <span class="primary-subject">Applications (stat.AP)</span>,
 <span class="primary-subject">Computation and Language (cs.CL)</span>,
 <span class="primary-subject">Computation and Language (cs.CL)</span>,
 <span class="primary-subject">Computation and Language (cs.CL)</span>,
 <span class="primary-subject">Computation and Language (cs.CL)</span>,
 <span class="primary-subject">Applications (stat.AP)</span>,
 <span class="primary-subject">Applications (stat.AP)</span>,
 <span class="primary-subject">Learning (cs.LG)</span>]